## Evaluating the pretrained speaker: MS COCO Captions evaluation

The following notebook is to set up / re-use public code for image caption evaluation following the pipeline suggested in the original paper on MS COCO Captions. Additionally, given the specifics of the downstream task (namely, sampling from the model and minimizing the CCE loss against that), a baseline for the validation PPL on sampled captions is computed as well.  

Due to installation difficulties, and because it was not a part of the original paper, the SPICE score computation is commented out in the cloned code and not performed in the present evaluation. 

#### Utils
In order to compute the standard image caption evaluation metrics, the code provided in [this](https://github.com/daqingliu/coco-caption) repo is used. Since it requires the results to be formatted in a specific syntax, script below performs some utility mapping of validation annotation IDs to  validation image IDs. 

--> i need to produce {'image_id': XXX, 'caption': 'lower cased string'} items when validating the model. When iterating over items with my data loader, I get annotation IDs. So i need to map ann IDs to img IDs. I can do that via th COCO.loadAnns(annIds) and then retrieve 'image_id'.

In [1]:
# reproducing the desired results format
import json
import torch
from pycocotools.coco import COCO
import math
import pandas as pd
from torchvision import transforms
import os
import sys
import torch.nn as nn
import numpy as np

In [2]:
# just creating a file for my entire val split
val_ids = torch.load("val_split_IDs_from_COCO_train.pt")
print(len(val_ids))
coco = COCO("../../../data/train/annotations/captions_train2014.json")
val_imgIDs = [coco.loadAnns(i)[0]['image_id'] for i in val_ids]
print(len(val_imgIDs))
# torch.save(val_imgIDs, "val_split_imgIDs_from_COCO_train.pt")

264048
loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
264048


A further restriction is that there must be only one produced caption per image so the evaluation happens on
one annID per unique image only -- the metrics are computed relative to all 5 ground truth captions anyways, but they are retrieved within the pipeline. Below, the respective annotation ID and image ID files are created. 

In [18]:
# i also need to create a new test split which only contains one annotation per unique image
# get images i know werent used for pretraining
with open("imgID2annID.json", "r") as fp:
    f = json.load(fp)
imgIDs4val = list(f.keys())[30000:]

ann4val_unqIm = [f[i][0] for i in imgIDs4val]
# torch.save(torch.tensor(ann4val_unqIm), "val_split_annIDs_singular_from_COCO_train_tensor.pt")

In [2]:
# load unique data
val_ids = torch.load("val_split_annIDs_singular_from_COCO_train_tensor.pt").tolist()
print(len(val_ids))
coco = COCO("../../../data/train/annotations/captions_train2014.json")
val_imgIDs = [coco.loadAnns(i)[0]['image_id'] for i in val_ids]
print(len(val_imgIDs))
# torch.save(val_imgIDs, "val_split_imgIDs_singular_from_COCO_train.pt")

52783
loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
52783


#### Evaluation function
The wrapper below takes in a trained model and performs the evaluation on a set number of validation images (e.g., a set including the images used in the reference game, or images used neither in pretraining nor experiments). 

In [2]:
# import agent modules from the actual repo
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from agents.speaker import DecoderRNN
from utils.build_dataset import get_loader
from reference_game_utils.update_policy import clean_sentence
from utils.vocabulary import Vocabulary

In [4]:
from coco_caption.pycocotools.coco import COCO 
from coco_caption.pycocoevalcap.eval import COCOEvalCap # TODO in readme add point about renaming!
import skimage.io as io

import json
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

In [16]:
def evaluate_speaker(
    model_path: str,
    num_val_imgs: int,
    res_path: str,
    val_ppl_path: str,
    metrics_res_path: str,
    vocab_file: str,
    download_dir: str,
    val_file: str, 
    val_imgIDs_file: str,
    vocab_threshold: int = 25,
    batch_size: int = 1,
    embed_size: int = 512,
    visual_embed_size: int = 512,
    hidden_size: int = 512,
    decoding_strategy: str = "greedy",
) -> None:
    """
    Evaluate a pretrained speaker (image captioner) model for MS COCO by computing: 
    1) validation loss + perplexity given a paricular decoding strategy
    2) image captioning evaluation metrics from MS COCO Captions:
        BLEU, ROUGE, METEOR and CIDEr.
    All results including produced captions are saved.
    
    Arguments:
    ---------
    model_path: str
        Path to speaker model weights.
    num_val_imgs: int
        Number of validation images to be used.
    res_path: str
        Path and name of file where produced captions will be saved.
    val_ppl_path: str
        Path and name of file where batch-wise validation loss and PPL will be written to.
    metrics_res_path: str
        Path and name of file where COCO Captions metrics will be written to.
    vocab_file: str
        Path to vocab file.
    download_dir: str
        Directory with annotations.
    val_file: str
        Path to file holding UNIQUE per image annotation IDs from validation set.
    val_imgIDs_file: str
        Path to image IDs corresponding to the validation annotation IDs above.
    vocab_threshold: int = 25
        Minimal token count used in vocabulary construction.
    batch_size: int = 1
        Must be 1.
    embed_size: int = 512
        Dimensionality of embeddings. Must correspond to pretraining settings.
    visual_embed_size: int = 512
        Dimensionality of image vectors. Must correspond to pretraining settings.
    hidden_size: int = 512
        Dimensionality of the hidden layer. Must correspond to pretraining settings.
    decoding_strategy: str = "greedy"
        Decoding strategy to be used in sampling. 
        Available options: "greedy", "exp", "pure", "topk_temperature", "encoding".
        If "encode" is used, ground truth captions are just passed through the LSTM in training mode,
        no decoding is taking place.
    """
    assert batch_size == 1, "Only batch_size=1 evaluations are supported!"
    
    # data loader
    transform_test = transforms.Compose([transforms.Resize((224, 224)), 
                                         transforms.ToTensor(), \
                                         transforms.Normalize((0.485, 0.456, 0.406), \
                                                          (0.229, 0.224, 0.225))])
    data_loader_test = get_loader(transform=transform_test,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=True,
                         download_dir=download_dir, 
                         vocab_file=vocab_file,
                         dataset_path=val_file, 
                         num_imgs=num_val_imgs,
                         embedded_imgs=torch.load("../train_logs/COCO_train_ResNet_features_reshaped_dict.pt"),
                        )
    # add img IDs
    data_loader_test.dataset._img_ids_flat = torch.load(val_imgIDs_file)[:num_val_imgs]
    val_imgIDs = torch.load(val_imgIDs_file)[:num_val_imgs]
    
    vocab_size = len(data_loader_test.dataset.vocab)
    # load model
    decoder = DecoderRNN(
        embed_size,
        hidden_size,
        vocab_size,
        visual_embed_size,
    )
    decoder.load_state_dict(torch.load(model_path))
    decoder.eval()
    criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()
    hidden = decoder.init_hidden(batch_size)
    
    # instantiate results 
    results = []
    val_running_loss = 0.0
    val_running_ppl = 0.0
    losses_list = []
    ppl_list = []
    counter = 0
    total = 0
    
    num_steps = math.ceil(len(data_loader_test.dataset)/batch_size)
    
    softmax = nn.Softmax(dim = -1)
        
    for i in range(num_steps):
        counter += 1
        # manually construct indices to avoid duplications bc of length of examples or random repetitions
        indices = [(i, 0)]
        
        new_sampler = torch.utils.data.sampler.SubsetRandomSampler(indices=indices)
        data_loader_test.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        targets, distractors, target_features, distractor_features, target_captions, distractor_captions = next(iter(data_loader_test)) 
        print("target caption ", clean_sentence(target_captions, data_loader_test))
        both_images = torch.cat((target_features.unsqueeze(1), distractor_features.unsqueeze(1)), dim=1)
        # retrieve image IDs
        batch_img_ids = [val_imgIDs[i[0]] for i in indices]
        print("Image id ", batch_img_ids)
        
        max_seq_len = target_captions.shape[1]-1
        
        with torch.no_grad():
            # get prediction
            if decoding_strategy == "encoding":
                outputs, _ = decoder(both_images, target_captions, hidden)
                norm_outputs = softmax(outputs)
                _, captions_pred = torch.max(norm_outputs, dim = -1)
                
            else: 
                captions_pred, log_probs, outputs, entropies = decoder.sample(
                    both_images, 
                    max_sequence_length=max_seq_len, 
                    decoding_strategy=decoding_strategy
                )
            # transform to natural language
            nl_captions_pred = clean_sentence(captions_pred, data_loader_test)
            print("NL captions pred ", nl_captions_pred)
            # append to results list together with img ID
            for i, c in list(zip(batch_img_ids, nl_captions_pred)):
                
                if "end" in c.split(" "):
                    len_c = sum([1 for x in c.split(" ")[:c.split(" ").index("end")] if x != "end" ])
                else:
                    len_c = len(c.split(" "))            
                
                results.append({"image_id": i, "caption": " ".join(c.split()[:len_c])})
                
            # compute val PPL
            loss = criterion(outputs.transpose(1,2), target_captions[:, 1:]) 
            losses_list.append(loss.item())
            ppl = np.exp(loss.item())
            ppl_list.append(ppl)

            val_running_loss += loss.item()
            val_running_ppl += ppl
    
    print("Final average loss: ", val_running_loss / counter)
    print("Final average PPL: ", val_running_ppl / counter)
        
    # check if results dir exists
    os.makedirs("../../../data/speaker_eval_results/", exist_ok=True)
    
    # write out results file
    with open(res_path, "w") as f:
        json.dump(results, f)
    # write out validation PPLs
    df_out = pd.DataFrame({
        "loss": losses_list,
        "PPL": ppl_list,
    })
    df_out.to_csv(val_ppl_path)
    
    # now compute the evaluations, as proposed in the notebook from the repo referenced above 
    cocoRes = data_loader_test.dataset.coco.loadRes(res_path)
    # create cocoEval object by taking coco and cocoRes
    cocoEval = COCOEvalCap(data_loader_test.dataset.coco, cocoRes)

    cocoEval.params['image_id'] = cocoRes.getImgIds()

    # evaluate results
    cocoEval.evaluate()
    coco_metrics = cocoEval.eval.items()
    # construct out file 
    metrics_df = pd.DataFrame(cocoEval.eval, index = [0]).round(3)
    metrics_df.to_csv(
        metrics_res_path
    )
    print("Final coco metrics: ", coco_metrics)
    

In [11]:
# evaluate scheduled sampling speaker
evaluate_speaker(
    model_path="../models/decoder-coco-512dim-scheduled_sampling_wGreedyDecoding_k150-1.pkl",
    num_val_imgs=10,
    res_path="../../../data/speaker_eval_results/decoder_scheduled_sampling_wGreedyDecoding_k150-1_results.json",
    val_ppl_path="../../../data/speaker_eval_results/decoder_scheduled_sampling_wGreedyDecoding_k150-1_val.json",
    metrics_res_path="../../../data/speaker_eval_results/decoder_scheduled_sampling_wGreedyDecoding_k150-1_COCO_metrics.csv",
    vocab_file="vocab4000.pkl",
    download_dir="../../../data/train",
    val_file="val_split_annIDs_singular_from_COCO_train_tensor.pt", 
    val_imgIDs_file="val_split_imgIDs_singular_from_COCO_train.pt",
    batch_size=1,
    decoding_strategy="encoding",
)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 3700/3700 [00:00<00:00, 100936.10it/s]


target caption  ['Start a train is coming up on another train next to it . end']
Image id  [341245]
target caption  ['Start a passenger train moves along rails next to apartment buildings . end']
Image id  [521150]
target caption  ['Start this train was photographed from an aerial view . end']
Image id  [568955]
target caption  ['Start a train with a brown engine is on a train track . end']
Image id  [393602]
target caption  ['Start trains passing on tracks near city area on cloudy day . end']
Image id  [576543]
target caption  ['Start check the phone to see if unk has called . end']
Image id  [92639]
target caption  ['Start intersection street signs displaying plaza drive and unk avenue . end']
Image id  [521132]
target caption  ['Start a street sign on the edge of a curb has a sign with a monkey end']
Image id  [573988]
target caption  ['Start a road sign hangs from an old rusted pole . end']
Image id  [86831]
target caption  ['Start a traffic light next to a street sign that says un

In [17]:
# evaluate original teacher forced speaker
# evaluate_speaker(
#     model_path="../models/decoder-noEnc-prepend-512dim-4000vocab-rs1234-wEmb-cont-7.pkl",
#     num_val_imgs=1000,
#     res_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_greedyDecoding_results.json",
#     val_ppl_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_greedyDecoding_val.json",
#     metrics_res_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_greedyDecoding_COCO_metrics.csv",
#     vocab_file="vocab4000.pkl",
#     download_dir="../../../data/train",
#     val_file="val_split_annIDs_singular_from_COCO_train_tensor.pt", 
#     val_imgIDs_file="val_split_imgIDs_singular_from_COCO_train.pt",
#     batch_size=1,
#     decoding_strategy="greedy",
# )
# evaluate_speaker(
#     model_path="../models/decoder-noEnc-prepend-512dim-4000vocab-rs1234-wEmb-cont-7.pkl",
#     num_val_imgs=1000,
#     res_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_encoding_results.json",
#     val_ppl_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_encoding_val.json",
#     metrics_res_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_encoding_COCO_metrics.csv",
#     vocab_file="vocab4000.pkl",
#     download_dir="../../../data/train",
#     val_file="val_split_annIDs_singular_from_COCO_train_tensor.pt", 
#     val_imgIDs_file="val_split_imgIDs_singular_from_COCO_train.pt",
#     batch_size=1,
#     decoding_strategy="encoding",
# )
evaluate_speaker(
    model_path="../models/decoder-noEnc-prepend-512dim-4000vocab-rs1234-wEmb-cont-7.pkl",
    num_val_imgs=10,
    res_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_pureDecoding_results.json",
    val_ppl_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_pureDecoding_val.json",
    metrics_res_path="../../../data/speaker_eval_results/decoder_orig_teacher_forced_pureDecoding_COCO_metrics.csv",
    vocab_file="vocab4000.pkl",
    download_dir="../../../data/train",
    val_file="val_split_annIDs_singular_from_COCO_train_tensor.pt", 
    val_imgIDs_file="val_split_imgIDs_singular_from_COCO_train.pt",
    batch_size=1,
    decoding_strategy="greedy",
)


Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 3700/3700 [00:00<00:00, 112677.25it/s]

target caption  ['Start a train is coming up on another train next to it . end']
Image id  [341245]


NL captions pred  ['A train traveling down train tracks . end . end . end .']
target caption  ['Start a passenger train moves along rails next to apartment buildings . end']
Image id  [521150]
NL captions pred  ['A train traveling down tracks in a city . end end .']
target caption  ['Start this train was photographed from an aerial view . end']
Image id  [568955]
NL captions pred  ['A train traveling down the train tracks . end .']
target caption  ['Start a train with a brown engine is on a train track . end']
Image id  [393602]
NL captions pred  ['A train is coming down the tracks in the middle of a train']
target caption  ['Start trains passing on tracks near city area on cloudy day . end']
Image id  [576543]
NL captions pred  ['A train is coming down the tracks in a train station .']
target caption  ['Start check the phone to see if unk has called . end']
Image id  [92639]
NL captions pred  ['A man in a suit and tie sitting on a chair']
target caption  ['Start intersection street si